In [20]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [21]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,8.818490,False,17,1,42.0
1,8.141671,False,29,1,38.0
2,5.948072,True,38,1,38.0
3,8.838332,True,27,1,39.0
4,9.259415,True,28,1,38.0


In [22]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9991.000000,10000.000000,10000.000000,9881.000000
mean,7.279259,27.332300,1.033200,38.704888
std,1.317603,6.238122,0.188948,2.602659
min,0.661387,12.000000,1.000000,19.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.124034,32.000000,1.000000,40.000000
max,12.808857,53.000000,3.000000,47.000000


In [23]:
df['is_male'].value_counts()

True     5158
False    4842
Name: is_male, dtype: int64

In [24]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [25]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [26]:
data.head()

,is_male,mother_age,plurality,gestation_weeks
757,1,23,1,39.0
6544,1,19,1,38.0
3190,1,28,1,40.0
2064,0,32,1,40.0
4066,0,17,1,40.0


In [27]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [28]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

In [29]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                320       
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 2,433
Trainable params: 2,433
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Train on 6665 samples, validate on 741 samples
Epoch 1/10
6665/6665 [==============================] - 2s 338us/sample - loss: 1.4316 - mae: 0.9376 - mse: 1.4316 - val_loss: 1.3441 - val_mae: 0.9111 - val_mse: 1.3441
Epoch 2/10
6665/6665 [==============================] - 1s 158us/sample - loss: 1.3152 - mae: 0.8982 - mse: 1.3152 - val_loss: 1.4878 - val_mae: 0.9809 - val_mse: 1.4878
Epoch 3/10
6665/6665 [==============================] - 1s 196us/sample - loss: 1.3145 - mae: 0.8999 - mse: 1.3145 - val_loss: 1.6555 - val_mae: 1.0141 - val_mse: 1.6555
Epoch 4/10
6665/6665 [==============================] - 1s 183us/sample - loss: 1.2843 - mae: 0.8866 - mse: 1.2843 - val_loss: 1.2929 - val_mae: 0.9024 - val_mse: 1.2929
Epoch 5/10
6665/6665 [==============================] - 1s 206us/sample - loss: 1.2758 - mae: 0.8838 - mse: 1.2758 - val_loss: 1.2110 - val_mae: 0.8537 - val_mse: 1.2110
Epoch 6/10
6665/6665 [==============================] - 1s 165us/sample - loss: 1.2551 - mae: 0.8773 - 

In [32]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [33]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  7.904254
Actual val:  7.06140625186

Predicted val:  7.490322
Actual val:  6.2501051276999995

Predicted val:  7.6461263
Actual val:  6.87401332916

Predicted val:  8.463426
Actual val:  6.75055446244

Predicted val:  7.0039425
Actual val:  8.12623897732

Predicted val:  7.8833327
Actual val:  7.48248917228

Predicted val:  8.091368
Actual val:  9.25059651352

Predicted val:  4.9192123
Actual val:  4.18657835538

Predicted val:  8.5569935
Actual val:  8.12623897732

Predicted val:  7.5078683
Actual val:  6.8122838958



In [34]:
wit_data = pd.concat([x_test, y_test], axis=1)

In [35]:
def custom_predict(examples_to_infer):
    preds = model.predict(examples_to_infer)
    return preds

In [36]:
config_builder = (WitConfigBuilder(wit_data[:500].values.tolist(), data.columns.tolist() + ['weight_pounds'])
  .set_custom_predict_fn(custom_predict)
  .set_target_feature('weight_pounds')
  .set_model_type('regression'))
WitWidget(config_builder, height=800)

WitWidget(config={'model_type': 'regression', 'label_vocab': [], 'feature_names': ['is_male', 'mother_age', 'p…